# Working notebooks

In [ ]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

In [ ]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
import numpy as np
from IPython.display import display, Markdown, FileLink, clear_output

In [ ]:
from IPython.display import Javascript
import glob as glob
import nbformat as nbf

label_layout = ipw.Layout(width='300px')

##########
pfiles = ['.protectedFiles.txt' , '../.protectedFiles.txt']
for fff in pfiles:
    if os.path.isfile(fff):
        with open(fff) as f:
            protectedFiles = f.read().splitlines()
##########
def launchNotebook(filename):
    text = "   var name_of_the_notebook = '" + filename + "'"
    vv="""
    var url = window.location.href.split('/')
    var newurl = url[0] + '//'
    for (var i = 1; i < url.length - 1; i++) {
        console.log(url[i], newurl)
        newurl += url[i] + '/'
    }
    newurl += name_of_the_notebook
    window.open(newurl)
    """
    text = text + vv
    display(Javascript(text))

def openNewNotebook(btn):
    if os.path.exists(notebookeName.value):
        print("Filename exists - Please select a different name")
        return

    nb = nbf.v4.new_notebook()
    text = """# Click 'Edit App' to start coding"""

    code = """\
# python packages
import pandas as pd # Dataframes and reading CSV files
import numpy as np # Numerical libraries
import matplotlib.pyplot as plt # Plotting library
from lmfit import Model # Least squares fitting library
from scipy.optimize import curve_fit # Alternative curve fittting library"""

    nb['cells'] = [nbf.v4.new_markdown_cell(text),
                   nbf.v4.new_code_cell(code)]

    if notebookeName.value in protectedFiles or notebookeName.value in listOfFiles:
        print("File already exists, select a different filename")
    else:
        with open(notebookeName.value, 'w') as f:
            nbf.write(nb, f)
        launchNotebook(notebookeName.value)

##########
listOfFiles = []
files = glob.glob1("./","*.ipynb")
for f in files:
    if f in protectedFiles:
        continue
    listOfFiles.append(f)

def dropdown_filesHandler(change):
    for i in range(0,len(listOfFiles)):
        if listOfFiles[i] == change.new:
            oldNotebookeName[0] = listOfFiles[i]

def createMenuFiles(data):
    option_list = ["Choose one"]
    option_list.extend(data)

    dropdown = ipw.Dropdown(description="", options=option_list, layout=ipw.Layout(width="300px"))
    dropdown.observe(dropdown_filesHandler, names='value')

    return dropdown

##########
oldNotebookeName = ["None"]
def openOldNotebook(btn):
    if oldNotebookeName[0] == "None":
        print("Please select a filename")
    elif oldNotebookeName[0] in protectedFiles:
        print("Please select a different filename")
    else:
        launchNotebook(oldNotebookeName[0])

##########
actions0 = []

notebookeName = ipw.Text("Empty.ipynb")

btn_new = ipw.Button(description="Create a new notebook", layout=label_layout)
btn_new.on_click(openNewNotebook)

btn_old = ipw.Button(description="Open an old notebook", layout=label_layout)
btn_old.on_click(openOldNotebook)

actions0.append(ipw.HBox([btn_new,notebookeName]))
actions0.append(ipw.HBox([btn_old,createMenuFiles(listOfFiles)]))
ipw.VBox(actions0)

# Bomb Calorimetry Virtual Experiment

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

In [ ]:
with open(".lab.json") as infile:
    jsdata = json.load(infile)

params = jsdata["cal1"]

In [ ]:
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

params["roomT"] = random.gauss(298,2)
params["slope"] = (2*random.random()) / params["nTime"]

# sample properties
params["mass"] = 0.0
params["mM"] = 0. # molar mass mole/g
params["n1"] = 0  # moles of CO2 per mole of substance burnt
params["n2"] = 0  # moles of H2O per mole of substance burnt
params["dn"] = 0  # change in moles of gas
params["dH"] = 0

In [ ]:
w = ipw.Dropdown(
    options=['None',
             'Benzoic acid', 
             'Sucrose', 
             'Naphthalene'],
    value='None',
    description='Sample:'
)

def sample_select(change):
    if change['new'] == "Benzoic acid":
        params["mM"] = 122.123
        params["n1"] = 7 
        params["n2"] = 3 
        params["dn"] = 7-15/2
        params["dH"] = -386000
        
    if change['new'] == "Sucrose":
        params["mM"] = 342.3
        params["n1"] = 12 
        params["n2"] = 11 
        params["dn"] = 0
        params["dH"] = -2221200

    if change['new'] == "Naphthalene":
        params["mM"] = 128.17
        params["n1"] = 10 
        params["n2"] = 4 
        params["dn"] = 10 - 12
        params["dH"] = -77000 

w.observe(sample_select)

In [ ]:
def run_experiment():
    
    # Error handing
    if float(params["mM"]) < 0.1:
        with out_Error:
            print("Please select a sample to use in the experiment")
        raise StopExecution

    if params["mass"] < 0.1:
        with out_Error:
            print("Please prepare a tablet first")
        raise StopExecution

    moles = params["mass"] / 1000 / params["mM"]

    # combustion enthalpy
    # nH{co2} + mH{h2o} - H = DcH
    DcH = params["n1"]*params["dH_co2"] + params["n2"]*params["dH_h2o"] - params["dH"]
    
    dH = DcH * moles
    dnrt = moles * params["R"] * params["roomT"] * params["dn"] 
    dU = dH - dnrt
    deltaT = -dU / params["cc"] #+ params["roomT"] 
   
    res = pd.DataFrame(columns=["Time [min]" , "T [K]"])
    for i in range(0, params["nTime"]):
        var_list = []
        var_list.append(i/10)
        tempOut = params["roomT"] + random.gauss(0,params["error"]) + params["slope"] * i
        if (i > params["mixTime"]):
            tempOut = tempOut + deltaT * (1-math.exp(-(i-params["mixTime"])/params["relaxTime"] ))
            
        var_list.append(tempOut)
        res.loc[len(res)] = var_list
    
    res.to_csv(respath.value, index=False)
    local_file = FileLink(respath.value, result_html_prefix="Click here to download: ")
    with out_P:
        display(local_file)
        display(res.tail(params["nTime"]))


In [ ]:
out_Error = ipw.Output()
out_P = ipw.Output()
out_T = ipw.Output()

# output filename
fileName = "results.csv"
respath = ipw.Text(fileName)

def reset(btn):
#     if os.path.exists(respath.value):
#         os.remove(respath.value)
    
    with out_Error:
        out_Error.clear_output()
        
    with out_P:
        out_P.clear_output()
        
    with out_T:
        out_T.clear_output()
    
    clear_output()
    create_ipw()

def calc(btn):
    if os.path.exists(respath.value):
        os.remove(respath.value)

    out_P.clear_output()
    run_experiment()

def make_tablet(btn):
    out_T.clear_output()

    params["mass"] = random.gauss(1000,100)
    
    with out_T:
        print("Mass of the tablet (mg) : %.2f " % params["mass"])
        

label_layout = ipw.Layout(width='300px')

# interactive buttons ---
btn_calc = ipw.Button(description="Perform Experiment", layout=label_layout)
btn_calc.on_click(calc)

btn_reset = ipw.Button(description="Reset Experiment", layout=label_layout)
btn_reset.on_click(reset)

btn_tab = ipw.Button(description="Make tablet", layout=label_layout)
btn_tab.on_click(make_tablet)

# ---
# reset(btn_reset)


# -- output widgets
def create_ipw():
    rows = []
    
    rows.append(ipw.HBox([ipw.Label('Output filename : ',layout=label_layout),respath]))
    rows.append(ipw.HBox([w]))
    rows.append(ipw.HBox([btn_tab]))
    rows.append(ipw.HBox([out_T]))
    rows.append(ipw.HBox([btn_calc,btn_reset]))
    rows.append(ipw.HBox([out_Error]))
    rows.append(ipw.HBox([out_P]))
    display(ipw.VBox(rows))

create_ipw()
